<a href="https://colab.research.google.com/github/mtwenzel/image-video-understanding/blob/master/Session_1_Learn_Single_Convolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Let's learn one convolution

In this experiment, we create a dataset by filtering MNIST with a pre-defined sharpen kernel. From the data, we train a network that should recover the kernel.

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Conv2D, Dropout, Dense
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam, SGD
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np


(x_train, y_train), (x_test, y_test) = mnist.load_data()

# reduce data by factor 10 / 20 for fast execution during course
x_train = x_train[::10]
y_train = y_train[::10]
x_test = x_test[::20]
y_test = y_test[::20]

# verify resulting array shapes
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
# Create a model to apply a hand-crafted filter.
model = Sequential()
model.add(InputLayer(input_shape=(28,28,1)))
model.add(Conv2D(filters=1, kernel_size=(3,3), padding='same', activation='linear'))
model.compile(loss='mse', optimizer=opt)
model.summary()

In [ ]:
w = np.asarray([np.asarray([
    [[[0]],[[-1]],[[0]]],
    [[[-1]],[[5]],[[-1]]],
    [[[0]],[[-1]],[[0]]]
    ], dtype=object),
    np.asarray([0])], dtype=object)
model.layers[0].set_weights(w)

In [ ]:
# Forward predict with the human-made "kernel" to create training data
y_train_images = model.predict(x_train)
y_train_images.shape, x_train.shape

In [ ]:
# One training example input
plt.imshow(x_train[600], cmap='gray')

In [ ]:
# One training example output (== filtered image)
plt.imshow(y_train_images[600].squeeze(), cmap='gray')

In [ ]:
opt = SGD(learning_rate=0.000009) # The training is extremely sensitive to the learning rate. Play with it.
#opt = Adam(learning_rate=0.005)  # Adam is much more stable, but converges much slower on such a simple problem.

model_t = Sequential()
model_t.add(InputLayer(input_shape=(28,28,1)))
model_t.add(Conv2D(filters=1, kernel_size=(3,3), padding='same', activation='linear'))
model_t.compile(loss='mse', optimizer=opt)
model_t.summary()

In [ ]:
history = model_t.fit(x_train, y_train_images, epochs=10)

In [ ]:
plt.plot(history.history['loss'])
plt.show()

In [ ]:
# Print the weights of the learned filter.
print(model_t.layers[0].get_weights())

In [ ]:
plt.imshow(y_train_images[600].squeeze(), cmap='gray')

In [ ]:
model_t_predicted = model_t.predict(x_train)
plt.imshow(model_t_predicted[600].squeeze(), cmap='gray')

In [ ]:
# A subtraction image between the desired and achieved output.
plt.imshow(y_train_images[600].squeeze()-model_t_predicted[600].squeeze(), cmap='gray')
print("Range of error: ", np.max(y_train_images[600].squeeze()-model_t_predicted[600].squeeze()), np.min(y_train_images[600].squeeze()-model_t_predicted[600].squeeze()))